In [13]:
import pandas as pd
df = pd.read_csv('../data/sdwpf_2001_2112_full.csv')
df.columns

Index(['TurbID', 'Tmstamp', 'Wspd', 'Wdir', 'Etmp', 'Itmp', 'Ndir', 'Pab1',
       'Pab2', 'Pab3', 'Prtv', 'T2m', 'Sp', 'RelH', 'Wspd_w', 'Wdir_w', 'Tp',
       'Patv'],
      dtype='object')

In [28]:
features = df.columns.drop(['TurbID', 'Tmstamp'])
mask = df.isna().any(axis=1)
bad_timestamps = df.loc[mask, 'Tmstamp']
df2 = df[~df['Tmstamp'].isin(bad_timestamps)]
df2 = df2.sort_values(by=['TurbID', 'Tmstamp']).reset_index(drop=True)

In [43]:
import numpy as np


turbine_data = []
for turb_id, group in df2.groupby('TurbID'):
    arr = group[features].values.T
    turbine_data.append(arr)

result_array = np.stack(turbine_data)
result_array = result_array.astype(np.float32)[:, :, :]
print(result_array.shape)

(134, 16, 60139)


In [45]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from net import gtnet  # Убедись, что файл net.py доступен

# Параметры
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_nodes = 134
in_dim = 16
seq_length = 8
out_dim = 2
total_time = 1000
batch_size = 128
epochs = 20
learning_rate = 0.001

# СИНТЕТИЧЕСКИЕ ДАННЫЕ (замени на свои)
np.random.seed(42)
data = result_array  # (134, 16, 60139)

# print(data)

# 👇 Разбиваем на обучающие примеры
def create_dataset(data, seq_len, pred_len):
    x_list, y_list = [], []
    for t in range(data.shape[2] - seq_len - pred_len + 1):
        x = data[:, :, t:t+seq_len]
        y = data[:, :, t+seq_len:t+seq_len+pred_len]
        x_list.append(x.transpose(1, 0, 2))  # -> (in_dim, num_nodes, seq_len)
        y_list.append(y.transpose(1, 0, 2))  # -> (in_dim, num_nodes, pred_len)

    # Преобразуем списки в numpy.ndarray перед созданием тензоров
    return torch.from_numpy(np.array(x_list)).float(), torch.from_numpy(np.array(y_list)).float()

X, Y = create_dataset(data, seq_length, out_dim)
# print("Dataset shape:", X.shape, Y.shape)

# 🔀 Разделение на train/test (80%/20%)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

# 📦 DataLoader'ы
train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
test_dataset = torch.utils.data.TensorDataset(X_test, Y_test)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 🤖 Инициализация модели
model = gtnet(
    gcn_true=True,
    buildA_true=True,
    gcn_depth=2,
    num_nodes=num_nodes,
    device=device,
    dropout=0.3,
    subgraph_size=5,
    node_dim=10,
    dilation_exponential=2,
    conv_channels=32,
    residual_channels=32,
    skip_channels=64,
    end_channels=128,
    seq_length=seq_length,
    in_dim=in_dim,
    out_dim=out_dim,
    layers=3
).to(device)

# 🔍 Оптимизатор и функция потерь
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.MSELoss()

# 🔁 Цикл обучения с валидацией
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch_x, batch_y in train_loader:
        batch_x = batch_x.to(device)
        batch_y = batch_y.to(device)

        optimizer.zero_grad()
        out = model(batch_x)  # [B, out_dim, num_nodes, 1]

        # Подготовка target
        target = batch_y.permute(0, 3, 2, 1)  # [B, out_dim, num_nodes, in_dim]
        target = target[..., 0]               # Берём только первый признак для сравнения

        loss = loss_fn(out.squeeze(-1), target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    # 🧪 Валидация
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            out = model(batch_x)
            target = batch_y.permute(0, 3, 2, 1)[..., 0]
            loss = loss_fn(out.squeeze(-1), target)
            val_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, "
          f"Train Loss: {total_loss / len(train_loader):.4f}, "
          f"Val Loss: {val_loss / len(test_loader):.4f}")

KeyboardInterrupt: 

In [1]:
print(device)

NameError: name 'device' is not defined

In [ ]:
import torch

if torch.cuda.is_available():
    print("GPU доступна!")
    print(f"Название GPU: {torch.cuda.get_device_name(0)}")
    print(f"Количество GPU: {torch.cuda.device_count()}")
else:
    print("GPU не доступна, используется CPU")